*I am taking the **IMDB dataset** for applying LSTM model. It will try to do the sentiment analysis from the reviews in the dataset.*

In [107]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk

### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout


import warnings
warnings.filterwarnings('ignore')

In [108]:
data=pd.read_csv("D:\Beinex\Python\Dataset-Kaggle\IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [109]:
data=data[0:10000] #selecting only first 10000 datapoints for less usage of comp. resources
data=data.astype(str)

In [110]:
data=data.replace({'sentiment':{'positive':1,'negative':0}})

In [111]:
x=data['review']
Y=data['sentiment']
print(x,Y)

0       One of the other reviewers has mentioned that ...
1       A wonderful little production. <br /><br />The...
2       I thought this was a wonderful way to spend ti...
3       Basically there's a family where a little boy ...
4       Petter Mattei's "Love in the Time of Money" is...
                              ...                        
9995    Fun, entertaining movie about WWII German spy ...
9996    Give me a break. How can anyone say that this ...
9997    This movie is a bad movie. But after watching ...
9998    This is a movie that was probably made to ente...
9999    Smashing film about film-making. Shows the int...
Name: review, Length: 10000, dtype: object 0       1
1       1
2       1
3       0
4       1
       ..
9995    1
9996    0
9997    0
9998    0
9999    1
Name: sentiment, Length: 10000, dtype: int64


In [112]:
from nltk.tokenize import word_tokenize
x=x.apply(word_tokenize)
print(x)

0       [One, of, the, other, reviewers, has, mentione...
1       [A, wonderful, little, production, ., <, br, /...
2       [I, thought, this, was, a, wonderful, way, to,...
3       [Basically, there, 's, a, family, where, a, li...
4       [Petter, Mattei, 's, ``, Love, in, the, Time, ...
                              ...                        
9995    [Fun, ,, entertaining, movie, about, WWII, Ger...
9996    [Give, me, a, break, ., How, can, anyone, say,...
9997    [This, movie, is, a, bad, movie, ., But, after...
9998    [This, is, a, movie, that, was, probably, made...
9999    [Smashing, film, about, film-making, ., Shows,...
Name: review, Length: 10000, dtype: object


In [113]:
type(x)
x=x.astype(str)

In [114]:
import re
x= x.apply(str.lower)
print(x)

0       ['one', 'of', 'the', 'other', 'reviewers', 'ha...
1       ['a', 'wonderful', 'little', 'production', '.'...
2       ['i', 'thought', 'this', 'was', 'a', 'wonderfu...
3       ['basically', 'there', "'s", 'a', 'family', 'w...
4       ['petter', 'mattei', "'s", '``', 'love', 'in',...
                              ...                        
9995    ['fun', ',', 'entertaining', 'movie', 'about',...
9996    ['give', 'me', 'a', 'break', '.', 'how', 'can'...
9997    ['this', 'movie', 'is', 'a', 'bad', 'movie', '...
9998    ['this', 'is', 'a', 'movie', 'that', 'was', 'p...
9999    ['smashing', 'film', 'about', 'film-making', '...
Name: review, Length: 10000, dtype: object


In [115]:
x = x.apply(lambda x: re.sub(r'[^\w\s]', '', x))
print(x)

0       one of the other reviewers has mentioned that ...
1       a wonderful little production   br    br   the...
2       i thought this was a wonderful way to spend ti...
3       basically there s a family where a little boy ...
4       petter mattei s  love in the time of money  is...
                              ...                        
9995    fun  entertaining movie about wwii german spy ...
9996    give me a break  how can anyone say that this ...
9997    this movie is a bad movie  but after watching ...
9998    this is a movie that was probably made to ente...
9999    smashing film about filmmaking  shows the inte...
Name: review, Length: 10000, dtype: object


In [116]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words("english")
x=x.apply(lambda x: ' '.join(w for w in x.split() if w not in stop_words))
print(x)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ginus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production br br filming tech...
2       thought wonderful way spend time hot summer we...
3       basically family little boy jake thinks zombie...
4       petter mattei love time money visually stunnin...
                              ...                        
9995    fun entertaining movie wwii german spy julie a...
9996    give break anyone say good hockey movie know m...
9997    movie bad movie watching endless series bad ho...
9998    movie probably made entertain middle school ea...
9999    smashing film filmmaking shows intense strange...
Name: review, Length: 10000, dtype: object


In [117]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer as lm
x=[lm().lemmatize(word) for word in x]
len(x)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ginus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ginus\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


10000

In [118]:
from nltk.stem import PorterStemmer as ps
x=[ps().stem(word) for word in x]
len(x)

10000

In [119]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
x_vector=vectorizer.fit(x)
print("Vocabulary: ", vectorizer.vocabulary_)

Vocabulary:  {'one': 39272, 'reviewers': 46561, 'mentioned': 34938, 'watching': 60482, 'oz': 40230, 'episode': 18551, 'hooked': 26516, 'right': 46801, 'exactly': 19054, 'happened': 25021, 'br': 7431, 'first': 20819, 'thing': 55831, 'struck': 53539, 'brutality': 7957, 'unflinching': 58431, 'scenes': 48410, 'violence': 59829, 'set': 49548, 'word': 61733, 'go': 23380, 'trust': 57436, 'show': 50248, 'faint': 19758, 'hearted': 25468, 'timid': 56266, 'pulls': 44045, 'punches': 44083, 'regards': 45605, 'drugs': 16946, 'sex': 49630, 'hardcore': 25068, 'classic': 10629, 'use': 59088, 'called': 8632, 'nickname': 37906, 'given': 23190, 'oswald': 39740, 'maximum': 34392, 'security': 49011, 'state': 52824, 'penitentary': 41188, 'focuses': 21285, 'mainly': 33516, 'emerald': 18009, 'city': 10532, 'experimental': 19357, 'section': 49000, 'prison': 43380, 'cells': 9489, 'glass': 23241, 'fronts': 22040, 'face': 19675, 'inwards': 28874, 'privacy': 43391, 'high': 25935, 'agenda': 2062, 'em': 17936, 'home'

In [120]:
vector =vectorizer.transform(x)
X=(vector.toarray())
X.shape

(10000, 62763)

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y,train_size=0.8,test_size=0.2,random_state=33)

In [128]:
# Pad sequences to a fixed length
max_sequence_length=130
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_sequence_length)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_sequence_length)

In [136]:
# Set the vocabulary size and maximum sequence length
vocab_size = 65000
# Define the LSTM model
embedding_dim = 128
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

In [137]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='AdaGrad', metrics=['accuracy'])


In [138]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 130, 128)          8320000   
                                                                 
 lstm_14 (LSTM)              (None, 130, 32)           20608     
                                                                 
 dropout_4 (Dropout)         (None, 130, 32)           0         
                                                                 
 lstm_15 (LSTM)              (None, 64)                24832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 8365505 (31.91 MB)
Trainable params: 8365505 (31.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [139]:
model.fit(X_train,Y_train, batch_size=30, epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
267/267 [==============================] - 25s 81ms/step - loss: 0.6932 - accuracy: 0.4934 - val_loss: 0.6931 - val_accuracy: 0.5005
Epoch 2/10
267/267 [==============================] - 28s 104ms/step - loss: 0.6932 - accuracy: 0.4986 - val_loss: 0.6932 - val_accuracy: 0.5005
Epoch 3/10
267/267 [==============================] - 21s 81ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6932 - val_accuracy: 0.5005
Epoch 4/10
267/267 [==============================] - 26s 97ms/step - loss: 0.6932 - accuracy: 0.5034 - val_loss: 0.6932 - val_accuracy: 0.5005
Epoch 5/10
267/267 [==============================] - 35s 133ms/step - loss: 0.6932 - accuracy: 0.5027 - val_loss: 0.6932 - val_accuracy: 0.5005
Epoch 6/10
267/267 [==============================] - 31s 115ms/step - loss: 0.6931 - accuracy: 0.5009 - val_loss: 0.6932 - val_accuracy: 0.5005
Epoch 7/10
267/267 [==============================] - 33s 122ms/step - loss: 0.6932 - accuracy: 0.5036 - val_loss: 0.6932 - val_accur

In [140]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test,Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 3s 51ms/step - loss: 0.6932 - accuracy: 0.5005
Test Loss: 0.6931614875793457
Test Accuracy: 0.5005000233650208


In [142]:
# Make predictions on the test data
Y_pred= model.predict(X_test)

# Convert the predicted probabilities to class labels
Y_pred= np.round(Y_pred).flatten()

# Print the predicted labels
print("Predicted Labels:", Y_pred)


63/63 [==============================] - 1s 22ms/step
Predicted Labels: [1. 1. 1. ... 1. 1. 1.]


In [143]:
from sklearn.metrics import classification_report
cr=classification_report(Y_test,Y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       999
           1       0.50      1.00      0.67      1001

    accuracy                           0.50      2000
   macro avg       0.25      0.50      0.33      2000
weighted avg       0.25      0.50      0.33      2000



*Due to the time constarints and excessive computer processing time, I limited this project to 50% accuracy.*

*Rather than improving the performance of the model, I count this as a learning of the LSTM algorithm process*
*From the classification report it is undestood that, the model couldn't predict any of the 'negative' sentiment based on reviews.*
*The model needs further tuning.*

*Due to the time constraints, I limited the optimizers (adagrad, adam, adamax)*
*Also, I believe, I have to develop mored clarity in calculation of embedding size, vocab size, max_sequence length etc..*